In [1]:
import pymongo

In [2]:
conex = pymongo.MongoClient()
conex.list_database_names()

['BDClientes',
 'DBPractica',
 'DBProvincias',
 'DBSample',
 'admin',
 'config',
 'local']

In [3]:
db = conex.DBPractica
db.list_collection_names()

['autores', 'documentos']

# Consulta 1

In [39]:
%time
author_name = "Juerg Tschirren"
query1 = db.documentos.find({"authors": author_name}, {"_id": 0, "title": 1})
print('El listado de publicaciones de {} es el siguiente: \n {}'.format(author_name, list(query1)))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
El listado de publicaciones de Juerg Tschirren es el siguiente: 
 [{'title': "Extraction of Airways From CT (EXACT'09)."}, {'title': 'Automated Analysis of Doppler Ultrasound Velocity Flow Diagrams.'}, {'title': 'Intrathoracic airway trees: segmentation and airway morphology analysis from low-dose CT scans.'}, {'title': 'Matching and anatomical labeling of human airway tree.'}, {'title': 'Quantitative analysis of pulmonary airway tree structures.'}, {'title': 'Determination of the envelope function (maximum velocity curve) in Doppler ultrasound flow velocity diagrams.'}, {'title': 'Quantitative analysis of three-dimensional tubular tree structures.'}, {'title': 'Pulmonary CT Image Analysis and Computer Aided Detection.'}, {'title': 'Segmentation, Skeletonization, and Branchpoint Matching - A Fully Automated Quantitative Evaluation of Human Intrathoracic Airway Trees.'}, {'title': 'Automated Nomenclature Labeling of the Bro

In [37]:
# Imprimimos por pantalla algunas estadísticas relativas a la consulta realizada
query1_stats = db.documentos.find({"authors": author_name}, {"_id": 0, "title": 1}).explain()
display(query1_stats["executionStats"])
query1_stats["queryPlanner"]

{'executionSuccess': True,
 'nReturned': 12,
 'executionTimeMillis': 3691,
 'totalKeysExamined': 0,
 'totalDocsExamined': 4873399,
 'executionStages': {'stage': 'PROJECTION_SIMPLE',
  'nReturned': 12,
  'executionTimeMillisEstimate': 43,
  'works': 4873401,
  'advanced': 12,
  'needTime': 4873388,
  'needYield': 0,
  'saveState': 38073,
  'restoreState': 38073,
  'isEOF': 1,
  'transformBy': {'_id': 0, 'title': 1},
  'inputStage': {'stage': 'COLLSCAN',
   'filter': {'authors': {'$eq': 'Juerg Tschirren'}},
   'nReturned': 12,
   'executionTimeMillisEstimate': 41,
   'works': 4873401,
   'advanced': 12,
   'needTime': 4873388,
   'needYield': 0,
   'saveState': 38073,
   'restoreState': 38073,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 4873399}},
 'allPlansExecution': []}

{'plannerVersion': 1,
 'namespace': 'DBPractica.documentos',
 'indexFilterSet': False,
 'parsedQuery': {'authors': {'$eq': 'Juerg Tschirren'}},
 'winningPlan': {'stage': 'PROJECTION_SIMPLE',
  'transformBy': {'_id': 0, 'title': 1},
  'inputStage': {'stage': 'COLLSCAN',
   'filter': {'authors': {'$eq': 'Juerg Tschirren'}},
   'direction': 'forward'}},
 'rejectedPlans': []}

# Consulta 2

In [57]:
%time
# Using authors collection
author_name = "Juerg Tschirren"
query2 = db.autores.aggregate([{"$match": {"_id": author_name}}, {"$project": {"Total publicaciones": {"$size": "$publications"}}}])
for doc in query2:
    print(doc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 28.8 µs
{'_id': 'Juerg Tschirren', 'Total publicaciones': 12}


La opción explain no está soportada en combinación con aggregate en pymongo (https://api.mongodb.com/python/3.5.1/api/pymongo/collection.html). Para ver las estadísticas de una query es mejor en la shell de mongo. Lo máximo que creo que se puede sacar aquí es lo siguiente:

In [56]:
#query2_stats = db.autores.explain("executionStats").aggregate([{"$match": {"_id": "Juerg Tschirren"}}, {"$project": {"Total publicaciones": {"$size": "$publications"}}}]) #Esta query funciona en la mongo shell
query2_pipeline = [{"$match": {"_id": author_name}}, {"$project": {"Total publicaciones": {"$size": "$publications"}}}]
db.command('aggregate', 'autores', pipeline=query2_pipeline, explain=True)


{'stages': [{'$cursor': {'query': {'_id': 'Juerg Tschirren'},
    'fields': {'publications': 1, '_id': 1},
    'queryPlanner': {'plannerVersion': 1,
     'namespace': 'DBPractica.autores',
     'indexFilterSet': False,
     'parsedQuery': {'_id': {'$eq': 'Juerg Tschirren'}},
     'queryHash': '3A21D503',
     'planCacheKey': 'F053AE06',
     'winningPlan': {'stage': 'PROJECTION_SIMPLE',
      'transformBy': {'publications': 1, '_id': 1},
      'inputStage': {'stage': 'IDHACK'}},
     'rejectedPlans': []}}},
  {'$project': {'_id': True,
    'Total publicaciones': {'$size': ['$publications']}}}],
 'serverInfo': {'host': 'MBP-de-Manuel.home',
  'port': 27017,
  'version': '4.2.3',
  'gitVersion': '6874650b362138df74be53d366bbefc321ea32d4'},
 'ok': 1.0}

# Consulta 3:

In [59]:
%time
year = "2018"
publication_type = "article"
query3 = db.documentos.count_documents({"$and": [{"year": year}, {"type": publication_type}]})
print('El número de artículos en 2018 es: {}'.format(query3))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
El número de artículos en 2018 es: 179819


In [65]:
# Using aggregation framework
%time
query3_2 = db.documentos.aggregate([{"$match": {"$and": [{"year": year}, {"type": publication_type}]}},
                                  {"$count": "Total articulos 2018"}])
for doc in query3_2:
    print(doc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 30 µs
{'Total articulos 2018': 179819}


In [66]:
#query3_stats =  db.documentos.explain("executionStats").aggregate([{"$match": {"$and": [{"year": "2018"}, {"type": "article"}]}}, {"$count": "Total articulos 2018"}])
query3_pipeline = [{"$match": {"$and": [{"year": year}, {"type": publication_type}]}},
                                  {"$count": "Total articulos 2018"}]
db.command('aggregate', 'autores', pipeline=query3_pipeline, explain=True)

{'stages': [{'$cursor': {'query': {'$and': [{'year': '2018'},
      {'type': 'article'}]},
    'queryPlanner': {'plannerVersion': 1,
     'namespace': 'DBPractica.autores',
     'indexFilterSet': False,
     'parsedQuery': {'$and': [{'type': {'$eq': 'article'}},
       {'year': {'$eq': '2018'}}]},
     'queryHash': '65CBB061',
     'planCacheKey': '65CBB061',
     'winningPlan': {'stage': 'COLLSCAN',
      'filter': {'$and': [{'type': {'$eq': 'article'}},
        {'year': {'$eq': '2018'}}]},
      'direction': 'forward'},
     'rejectedPlans': []}}},
  {'$group': {'_id': {'$const': None},
    'Total articulos 2018': {'$sum': {'$const': 1}}}},
  {'$project': {'_id': False, 'Total articulos 2018': True}}],
 'serverInfo': {'host': 'MBP-de-Manuel.home',
  'port': 27017,
  'version': '4.2.3',
  'gitVersion': '6874650b362138df74be53d366bbefc321ea32d4'},
 'ok': 1.0}

# Consulta 4:

In [73]:
%time
query4 = db.autores.aggregate([{"$project": {"num_publications": {"$size": "$publications"}}},
                               {"$match": {"num_publications": {"$lt": 5}}},
                               {"$count": "Autores con menos de 5 publicaciones"}])
for doc in query4:
    print(doc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
{'Autores con menos de 5 publicaciones': 1935763}


In [ ]:
#query4_stats =  db.autores.explain("executionStats").aggregate([{"$project": {"num_publications": {"$size": "$publications"}}}, {"$match": {"num_publications": {"$lt": 5}}}, {"$count": "Autores con menos de 5 publicaciones"}])

# Consulta 5:

In [93]:
%time
query5 = db.autores.aggregate([{"$match": {"_id": {"$ne": None}}}, # Para filtrar las publicaciones sin autor
                               {"$addFields": {"Total documentos": {"$size": "$publications"}}},
                               {"$sort": {"Total documentos": -1}},
                               {"$limit": 10},
                               {"$project": {"Total documentos": 1}},
                               {"$lookup": {"from": "documentos", "localField": "_id", "foreignField": "authors", "as": "documents"}},
                               {"$unwind": "$documents"},
                               {"$match": {"documents.type": {"$in": ["article", "inproceedings"]}}},
                               {"$sortByCount": "$documents.type"}], allowDiskUse = True)

for doc in query5:
    print(doc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
{'_id': 'article', 'count': 6936}
{'_id': 'inproceedings', 'count': 6566}


In [ ]:
# query5_stats = db.autores.aggregate([{"$match": {"_id": {"$ne": null}}}, {"$addFields": {"Total documentos": {"$size": "$publications"}}}, {"$sort": {"Total documentos": -1}}, {"$limit": 10}, {"$project": {"Total documentos": 1}}, {"$lookup": {"from": "documentos", "localField": "_id", "foreignField": "authors", "as": "documents"}}, {"$unwind": "$documents"}, {"$match": {"documents.type": {"$in": ["article", "inproceedings"]}}}, {"$sortByCount": "$documents.type"}])

# Consulta 6:

In [78]:
%time
query6 = db.documentos.aggregate([{"$project": {"Numero autores": {"$size": "$authors"}}},
                                {"$group": {"_id": None, "Numero medio autores": {"$avg": "$Numero autores"}}},
                                {"$project": {"_id": 0}}])

for doc in query6:
    print(doc)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs
{'Numero medio autores': 3.06153138702577}


In [79]:
#query6_stats = db.documentos.explain("executionStats").aggregate([{"$project": {"Numero autores": {"$size": "$authors"}}}, {"$group": {"_id": null, "Numero medio autores": {"$avg": "$Numero autores"}}}, {"$project": {"_id": 0}}])

# Consulta 7:

In [81]:
%time
author_name = "Juerg Tschirren"
query7 = db.documentos.aggregate([{"$match": {"authors": author_name}},
                                      {"$unwind": "$authors"},
                                      {"$group": {"_id": None, "Autores": {"$addToSet": "$authors"}}},
                                      {"$project": {"_id": 0, "Coautores": {"$setDifference": ["$Autores", [author_name]]}}}])

for doc in query7:
    print(doc)

CPU times: user 5 µs, sys: 16 µs, total: 21 µs
Wall time: 23.1 µs
{'Coautores': ['Bram van Ginneken', 'Hiroko Kitaoka', 'Mathias Prokop', 'Ieneke Hartmann', 'Catalin I. Fetita', 'Jan Sijbers', 'Anthony P. Reeves', 'Pechin Lo', 'Joseph M. Reinhardt', 'Jesper Johannes Holst Pedersen', 'Benjamin Odry', 'Edwin J. R. van Beek', 'Anna Fabijanska', 'Yongsup Park', 'Reinhard Beichel', 'Jaesung Lee', 'Soumik Ukil', 'X. Zhang', 'Oliver Weinheimer', 'Pim A. de Jong', 'Eva M. van Rikxoort', 'Geoffrey McLennan', 'Silvia Born', 'David P. Naidich', 'Marco Feuerstein', 'Ronald M. Lauer', 'Rafael Wiemker', 'Eric A. Hoffman', 'Carlos S. Mendoza', 'Kensaku Mori', 'Milan Sonka', 'Ye Xu', 'Tarunashree Yavarna', 'Romulo Pinho', 'Christian Bauer 0001', 'Benjamin Irving', 'Kalman Palagyi', 'Margarete Ortner', 'Marleen de Bruijne']}


In [ ]:
# query7_stats = db.documentos.explain("executionStats").aggregate([{"$match": {"authors": "Juerg Tschirren"}}, {"$unwind": "$authors"}, {"$group": {"_id": null, "Autores": {"$addToSet": "$authors"}}}, {"$project": {"_id": 0, "Coautores": {"$setDifference": ["$Autores", ["Juerg Tschirren"]]}}}])

# Consulta 8:

In [83]:
%time
query8 = db.documentos.aggregate([{"$unwind": "$authors"},
                                        {"$group": {"_id": "$authors", "Primero": {"$min": "$year"}, "Ultimo": {"$max": "$year"}}},
                                        {"$addFields": {"Ultimo": {"$toInt": "$Ultimo"}, "Primero": {"$toInt": "$Primero"}}},
                                        {"$addFields": {"Edad": {"$subtract": ["$Ultimo", "$Primero"]}}},
                                        {"$sort": {"Edad": -1}},
                                        {"$limit": 5},
                                        {"$project": {"_id": 1, "Edad": 1}}], allowDiskUse = True)
                                        



for doc in query8:
    print(doc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 19.1 µs
{'_id': 'Alan M. Turing', 'Edad': 75}
{'_id': 'Claude E. Shannon', 'Edad': 71}
{'_id': 'Rudolf Carnap', 'Edad': 71}
{'_id': 'David Nelson', 'Edad': 70}
{'_id': None, 'Edad': 66}


In [ ]:
#query8_stats = db.documentos.explain("executionStats").aggregate([{"$unwind": "$authors"}, {"$group": {"_id": "$authors", "Primero": {"$min": "$year"}, "Ultimo": {"$max": "$year"}}}, {"$addFields": {"Ultimo": {"$toInt": "$Ultimo"}, "Primero": {"$toInt": "$Primero"}}}, {"$addFields": {"Edad": {"$subtract": ["$Ultimo", "$Primero"]}}}, {"$sort": {"Edad": -1}}, {"$limit": 5}, {"$project": {"_id": 1, "Edad": 1}}], {allowDiskUse: true})

# Consulta 9:

In [85]:
%time
query9 = db.documentos.aggregate([{"$unwind": "$authors"},
                                {"$group": {"_id": "$authors", "Primero": {"$min": "$year"}, "Ultimo": {"$max": "$year"}}},
                                {"$addFields": {"Ultimo": {"$toInt": "$Ultimo"}, "Primero": {"$toInt": "$Primero"}}},
                                {"$addFields": {"Edad": {"$subtract": ["$Ultimo", "$Primero"]}}},
                                {"$match": {"Edad": {"$lt": 5}}},
                                {"$count": "Numero de autores novatos"}], allowDiskUse = True)
                                 
    
for doc in query9:
    print(doc)    

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
{'Numero de autores novatos': 1848369}


In [ ]:
#query9_stats = db.documentos.explain("executionStats").aggregate([{"$unwind": "$authors"}, {"$group": {"_id": "$authors", "Primero": {"$min": "$year"}, "Ultimo": {"$max": "$year"}}}, {"$addFields": {"Ultimo": {"$toInt": "$Ultimo"}, "Primero": {"$toInt": "$Primero"}}}, {"$addFields": {"Edad": {"$subtract": ["$Ultimo", "$Primero"]}}}, {"$match": {"Edad": {"$lt": 5}}}, {"$count": "Numero de autores novatos"}], {allowDiskUse: true})

                                

# Consulta 10:

In [87]:
%time
query10 = db.documentos.aggregate([{"$project": {"_id": 1, "Articulo": {"$cond": [{"$eq": ["$type","article"]},1,0]}}}, 
                                  {"$group": {"_id": None, "Total Articulos": {"$sum": "$Articulo"}, "Total Documentos": {"$sum": 1}}}, 
                                  {"$project": {"_id": 0, "Porcentaje de articulos": {'$multiply':[{'$divide':["$Total Articulos","$Total Documentos"]},100]}}}])



for doc in query10:
    print(doc)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
{'Porcentaje de articulos': 45.89263058493672}


In [ ]:
#query10_stats = db.documentos.explain("executionStats").aggregate([{"$project": {"_id": 1, "Articulo": {"$cond": [{"$eq": ["$type","article"]},1,0]}}}, {"$group": {"_id": null, "Total Articulos": {"$sum": "$Articulo"}, "Total Documentos": {"$sum": 1}}}, {"$project": {"_id": 0, "Porcentaje de articulos": {'$multiply':[{'$divide':["$Total Articulos","$Total Documentos"]},100]}}}])